In [1]:
import os
import torch

try:
  from google.colab import drive
  drive.mount('/content/drive')

  os.chdir('/content/drive/MyDrive/GitHub/kornia')
except Exception:   # Not run on Google Colab
  import getpass
  if getpass.getuser() == 'vscode':       # Running in container
    os.chdir('/workspaces/kornia/')
  else:                                   # Running in virtual environment
    os.chdir(os.path.join(os.path.expanduser('~'), 'Documents/Git/kornia/'))

  print(os.getcwd())
  import sys
  sys.path.insert(0, os.getcwd())

/home/yhollander/Documents/Git/kornia


In [2]:
from kornia.nerf.camera_utils import parse_colmap_output, create_spline_curve
from kornia.nerf.rays import FocalAxisRay, sample_lengths, sample_ray_points

In [3]:
img_names, cameras = parse_colmap_output('./test/nerf/cameras.txt', './test/nerf/images.txt', 'cpu', torch.float32)

In [4]:
cameras_new = create_spline_curve(cameras, 50)

In [5]:
# Camera origins
o = cameras.origins().squeeze(-1)
o_new = cameras_new.origins().squeeze(-1)

print(o[0], o_new[0])

tensor([-5.5204, -2.2255,  1.1888]) tensor([-5.5204, -2.2255,  1.1888])


In [6]:
# cameras_new.extrinsics[0, 0:3, 0:3] = torch.eye(3)

# Focal axes for cameras
focal_axis_rays = FocalAxisRay(10., corner_rays=True)
focal_axis_rays.calc_ray_params(cameras_new)

lengths = sample_lengths(focal_axis_rays.origins.shape[0], 2, irregular=False, device='cpu', 
                         dtype=torch.float32)
points_3d = sample_ray_points(
    focal_axis_rays.origins,
    focal_axis_rays.directions,
    lengths,
)

In [8]:
import plotly.graph_objects as go
fig = go.Figure()
# fig.add_trace(go.Scatter3d(x=focal_axis_rays.origins[:, 0], y=focal_axis_rays.origins[:, 1], z=focal_axis_rays.origins[:, 2]))
fig.add_trace(go.Scatter3d(x=o_new[:, 0], y=o_new[:, 1], z=o_new[:, 2]))

# Focal lines
for i in range(points_3d.shape[0]):
    fig.add_trace(go.Scatter3d(x=points_3d[i, 0:2, 0], y=points_3d[i, 0:2, 1], z=points_3d[i, 0:2, 2], mode='lines'))
fig.show()

In [14]:
points_3d.shape

torch.Size([50, 2, 3])